In [48]:
import glob
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from matplotlib.colors import ListedColormap
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_moons, make_circles, make_classification
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.model_selection import cross_val_score

In [49]:
path = "/home/jorgher/Documents/faceRec/CFEE/"
files = sorted(glob.glob(path+'*.csv'))
files = files[:7]

In [50]:
path = "/home/jorgher/Documents/faceRec/CFEE/"
files = sorted(glob.glob(path+'*.csv'))
files = files[:7]
#emotions = ["neutral",'happy','sad','fearful','angry','surprised','disgusted']
emotions = [0,1,2,3,4,5,6]
dbs = []
for i,f in enumerate(files):
    db = pd.read_csv(f)
    db = db[['D1','D2','D3','D4','D5','D6','D7','A0','A1','A2','A3','A4','A5']]
    db['Emotions'] = emotions[i]
    dbs.append(db)

db = (pd.concat(dbs[i][:] for i in range(len(files))))

In [51]:
db.dtypes

D1          float64
D2          float64
D3          float64
D4          float64
D5          float64
D6          float64
D7          float64
A0          float64
A1          float64
A2          float64
A3          float64
A4          float64
A5          float64
Emotions      int64
dtype: object

In [52]:
features = list(db.columns[:13])
target = db['Emotions']
Etype={'neutral': 0, 'happy': 1,
        'sad': 2,'fearful': 3,'angry': 4,
        'surprised': 5,'disgusted': 6}

In [53]:
db_data = db[features]

In [54]:
db_target = db['Emotions']

In [74]:
X_train, X_test, y_train, y_test = train_test_split(db_data, db_target, test_size=0.2, random_state=0)

In [75]:
X_train.shape, y_train.shape

((1288, 13), (1288,))

In [76]:
X_test.shape, y_test.shape

((322, 13), (322,))

In [77]:
clf = SVC(kernel='linear', C=1).fit(X_train, y_train)

In [78]:
scores = cross_val_score(clf, db_data, db_target, cv=10)

In [79]:
scores

array([ 0.65217391,  0.59627329,  0.60869565,  0.63354037,  0.61490683,
        0.64596273,  0.70807453,  0.65217391,  0.65838509,  0.66459627])

In [83]:
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Accuracy: 0.64 (+/- 0.06)


In [84]:
from sklearn.model_selection import ShuffleSplit
n_samples = db_data.shape[0]
cv = ShuffleSplit(n_splits=3, test_size=0.3, random_state=0)
cross_val_score(clf, db_data, db_target, cv=cv)

array([ 0.61697723,  0.61904762,  0.61076605])

In [92]:
from sklearn import preprocessing
X_train, X_test, y_train, y_test = train_test_split(
    db_data, db_target, test_size=0.4, random_state=0)
scaler = preprocessing.StandardScaler().fit(X_train)
X_train_transformed = scaler.transform(X_train)
clf = SVC(C=1).fit(X_train_transformed, y_train)
X_test_transformed = scaler.transform(X_test)
clf.score(X_test_transformed, y_test)  

0.64130434782608692

In [93]:
from sklearn.pipeline import make_pipeline
clf = make_pipeline(preprocessing.StandardScaler(), SVC(C=1))
cross_val_score(clf, db_data, db_target, cv=cv)  

array([ 0.63975155,  0.63768116,  0.64182195])